In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import collections
import sqlite3
from pathlib import Path



In [ ]:
application = pd.read_csv('application_data.csv')

previous = pd.read_csv('previous_application.csv')


In [ ]:
application.info()

In [ ]:
application.head(5)

In [ ]:
previous.head()

In [ ]:

print(application['NAME_CONTRACT_TYPE'].value_counts())


In [ ]:
application.describe

In [ ]:
previous.info

In [ ]:
previous['NAME_CONTRACT_TYPE'].value_counts()

In [ ]:
previous['SK_ID_PREV'].duplicated().value_counts()

In [ ]:
previous['SK_ID_CURR'].duplicated().value_counts()

In [ ]:
application['NAME_CONTRACT_TYPE'].unique()

In [ ]:
previous['NAME_CONTRACT_TYPE'].unique()

In [ ]:
previous['WEEKDAY_APPR_PROCESS_START'].unique()

In [ ]:
previous['PRODUCT_COMBINATION'].unique()


In [ ]:
previous['PRODUCT_COMBINATION'].isna().value_counts()

In [ ]:
previous.groupby(by='WEEKDAY_APPR_PROCESS_START')['NAME_CONTRACT_STATUS'].describe()

In [ ]:
sns.heatmap(previous.corr(), annot = True)

In [56]:
spend.to_sql('spend', conn, if_exists='append', index = False)

In [59]:
c.execute('SELECT * FROM customers LIMIT 5').fetchall()

[(1, 'A1', 76, 'BANGALORE', 'Gold', 500000, 'C1', 'Self Employed'),
 (2, 'A2', 71, 'CALCUTTA', 'Silver', 100000, 'C2', 'Salaried_MNC'),
 (3, 'A3', 34, 'COCHIN', 'Platimum', 10000, 'C3', 'Salaried_Pvt'),
 (4, 'A4', 47, 'BOMBAY', 'Platimum', 10001, 'C4', 'Govt'),
 (5, 'A5', 56, 'BANGALORE', 'Platimum', 10002, 'C5', 'Normal Salary')]

In [60]:
c.execute('SELECT * FROM repayment LIMIT 5').fetchall()

[(None, 'A1', '12-Jan-04', 495414.75, None),
 (2, 'A1', '3-Jan-04', 245899.02, None),
 (3, 'A1', '15-Jan-04', 259490.06, None),
 (4, 'A1', '25-Jan-04', 437555.12, None),
 (5, 'A1', '17-Jan-05', 165972.88, None)]

In [61]:
c.execute('SELECT * FROM spend LIMIT 5').fetchall()

[(1, 'A1', '12-Jan-04', 'JEWELLERY', 485470.8),
 (2, 'A1', '3-Jan-04', 'PETRO', 410556.13),
 (3, 'A1', '15-Jan-04', 'CLOTHES', 23740.46),
 (4, 'A1', '25-Jan-04', 'FOOD', 484342.47),
 (5, 'A1', '17-Jan-05', 'CAMERA', 369694.07)]